In [2]:
import pandas as pd
from datetime import datetime, timedelta
from sqlalchemy import create_engine

In [3]:
file = "EPA_Info.csv"

In [4]:
epa_data_df = pd.read_csv(file)
epa_data_df.head()

,Date,Carbon Monoxide AQI Value,Main Pollutant,Site Name,Site ID,Source,Nitrogen Dioxide AQI Value,Main Pollutant.1,Site Name.1,Site ID.1,...,Sulfur Dioxide AQI Value,Main Pollutant.4,Site Name.4,Site ID.4,Source.4,PM10 AQI Value,Main Pollutant.5,Site Name.5,Site ID.5,Source.5
0,1/1/2017,11,NaN,Elizabeth Lab,34-039-0004,AQS,44,NaN,Elizabeth Lab,34-039-0004,...,4,NaN,PFIZER LAB SITE,36-005-0133,AQS,NaN,NaN,NaN,NaN,NaN
1,1/2/2017,13,NaN,Elizabeth,34-039-0003,AQS,43,NaN,Bayonne,34-017-0006,...,17,NaN,EISENHOWER PARK,36-059-0005,AQS,NaN,NaN,NaN,NaN,NaN
2,1/3/2017,7,NaN,Elizabeth,34-039-0003,AQS,42,NaN,Elizabeth Lab,34-039-0004,...,1,NaN,PFIZER LAB SITE,36-005-0133,AQS,NaN,NaN,NaN,NaN,NaN
3,1/4/2017,6,NaN,Jersey City,34-017-1002,AQS,27,NaN,Fort Lee Near Road,34-003-0010,...,3,NaN,Chester,34-027-3001,AQS,7.0,NaN,Newark Firehouse,34-013-0003,AQS
4,1/5/2017,6,NaN,Jersey City,34-017-1002,AQS,28,NaN,Jersey City,34-017-1002,...,3,NaN,QUEENS COLLEGE 2,36-081-0124,AQS,NaN,NaN,NaN,NaN,NaN


In [5]:
epa_data_df['Year'] = pd.DatetimeIndex(epa_data_df['Date']).year


In [6]:
new_epadata_df = epa_data_df[['Year', 'Sulfur Dioxide AQI Value',  'Nitrogen Dioxide AQI Value','Ozone AQI Value','Carbon Monoxide AQI Value','PM10 AQI Value', 'PM2.5 AQI Value']].copy()

new_epadata_df = new_epadata_df.rename(columns={'Sulfur Dioxide AQI Value': 'SO2','Nitrogen Dioxide AQI Value':'NO2', 'Ozone AQI Value':'O3', 'Carbon Monoxide AQI Value': 'CO','PM10 AQI Value': 'PM10', 'PM2.5 AQI Value': 'PM2.5'})

new_epadata_df.head()

,Year,SO2,NO2,O3,CO,PM10,PM2.5
0,2017,4,44,34,11,NaN,59
1,2017,17,43,33,13,NaN,60
2,2017,1,42,32,7,NaN,43
3,2017,3,27,32,6,7.0,46
4,2017,3,28,32,6,NaN,49


In [10]:
NY_2017 = new_epadata_df[new_epadata_df['Year'] == 2017]

NY_2017 = NY_2017.groupby('Year',as_index=False).mean()
NY_2017.insert(0, 'City', ['New York'], True)
NY_2017.style.format({'SO2': "{:,.2f}", 'NO2': '{:.2f}', 'O3': '{:.2f}', 'CO': '{:,.2f}', 'PM10': '{:,.2f}', 'PM2.5': '{:,.2f}'})



,City,Year,SO2,NO2,O3,CO,PM10,PM2.5
0,New York,2017,2.88,38.13,46.11,7.21,14.60,49.54


In [11]:
NY_2018 = new_epadata_df[new_epadata_df['Year'] == 2018]

NY_2018 = NY_2018.groupby('Year',as_index=False).mean()
NY_2018.insert(0, 'City', ['New York'], True)
NY_2018.style.format({'SO2': "{:,.2f}", 'NO2': '{:.2f}', 'O3': '{:.2f}', 'CO': '{:,.2f}', 'PM10': '{:,.2f}', 'PM2.5': '{:,.2f}'})

,City,Year,SO2,NO2,O3,CO,PM10,PM2.5
0,New York,2018,3.05,38.39,47.12,8.16,14.35,50.06


In [12]:
NY_2019 = new_epadata_df[new_epadata_df['Year'] == 2018]

NY_2019 = NY_2018.groupby('Year',as_index=False).mean()
NY_2019.insert(0, 'City', ['New York'], True)
NY_2019.style.format({'SO2': "{:,.2f}", 'NO2': '{:.2f}', 'O3': '{:.2f}', 'CO': '{:,.2f}', 'PM10': '{:,.2f}', 'PM2.5': '{:,.2f}'})

,City,Year,SO2,NO2,O3,CO,PM10,PM2.5
0,New York,2018,3.05,38.39,47.12,8.16,14.35,50.06


In [13]:
NY_2017.to_csv('NY_2017.csv', index=False)
NY_2018.to_csv('NY_2018.csv', index=False)
NY_2019.to_csv('NY_2019.csv', index=False)